## openpose

姿态模仿+面部修正

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


## import json file

In [2]:
promptJson = 'lora_roop_api.json'

with open(promptJson) as f:
    prompt = json.load(f)

In [3]:
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [4]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [6]:
# 子文件夹
subDir = "奶茶多多"

# prompts文件
promptsFile = 'descriptions.txt'
with open(promptsFile, encoding='utf-8') as f:
    prompt_list = f.readlines()

stylesFile = 'styles.txt'
with open(stylesFile, encoding='utf-8') as f:
    style_list = f.readlines()

style_list = [x.strip() for x in style_list]
# 删除所有为空的样式
style_list = [x for x in style_list if x != '']
# 删除所有不包含{input_text}的样式
style_list = [x for x in style_list if '{input_text}' in x]

# 设置图片保存路径
imageDir = f"e:/SharedDirectory/PoleDriver/remake/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

# 这里是要替换的人脸图片
faceReferenceImagePath = "c:/Users/BigHippo78/Documents/WeChat Files/highmore78/FileStorage/File/2023-10/奶茶多多/face.png"

['Colorful ink cascaded the canvas, photorealistic photo, studio lighting, {input_text},sony a7, 35mm, hyperrealistic, big depth of field, concept art, colors, hyperdetailed, hyperrealistic, (big depth of field), (moody lighting), (ambient light), ((cinematic)),', 'lace drawing,black & yellow, industrial, heavy duty, modular,{input_text}', 'science fiction, night time, volumetric light, dutch angle, cinematic angle,{input_text}', '3d Retro Game,{input_text},8-bit Color Palette, Pixel Texture, Pixel Art Style, Isometric Perspective, highly detailed, two tone lighting, vibrant colors, 100mm lens, 3d blender render,', 'cinematic lighting,{input_text}', 'natural, (ultrarealistic:2.0) photo, (highest quality), 4k, intricate details,{input_text}', 'alcohol ink paint by Agnes Ceceile, Marc Simonetti, Alberto Seveso, Jeremy Geddes, Gediminas Pranckeviciu, Igor Ozhinga, Thomas Kinkade, Jean Baptiste Mongue, James Christensen,{input_text} , cinematic lighting, highly detailed, sharp focus, iride

NameError: name 'os' is not defined

In [6]:

from PIL import Image
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 2

# 遍历提示词组
for i in range(0, len(prompt_list)):
        input_text = prompt_list[i]
        # 取..之间的内容作为prompt
        input_text = input_text.split(".")[1]
        
        promt_string = f"Bokeh,(TOK:1.5), {input_text}, she have black long hair,big breasts,big ass,ziprealism"  
        prompt['6']['inputs']['text'] = promt_string        
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = f"奶茶多多_{i}"
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue   
        
        # 下面是确定实际生成图片的大小，种子，以及保存图片信息
        # 保存图片的信息使用的是38号节点，也就是Save Image w/Metadata节点 
        prompt['36']['inputs']['batch_size'] = batch_size
        prompt['23']['inputs']['image'] = faceReferenceImagePath
        steps = prompt['3']['inputs']['steps']
        cfg = prompt['3']['inputs']['cfg']
        sampler_name = prompt['3']['inputs']['sampler_name']
        scheduler = prompt['3']['inputs']['scheduler']
        positive = prompt['6']['inputs']['text']
        negative = prompt['7']['inputs']['text']
        
        modelname = prompt['4']['inputs']['ckpt_name']
        basemodelname = parse_name(modelname)        
        ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname
        modelhash = calculate_sha256(ckpt_path)[:10]
        
        
        
        for j in range(3):
            seed = random.randint(0, 976242998978323)
            prompt['3']['inputs']['seed']= seed
            
            comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {896}x{1152}, Model hash: {modelhash}, Model: {basemodelname}, Version: ComfyUI"        
        
            print(f"生成图片的信息是：{comment}")

            originalImagName = f"奶茶多多_{i}_{j}"
            # 连接api，生成并保存图片
            start_time = time.time()
            
            images = get_images(ws, prompt)   
            saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
            end_time = time.time()
            print(f"耗时{end_time-start_time}秒")
        

奶茶多多_0已经存在，跳过
奶茶多多_1已经存在，跳过
奶茶多多_2已经存在，跳过
奶茶多多_3已经存在，跳过
奶茶多多_4已经存在，跳过
奶茶多多_5已经存在，跳过
奶茶多多_6已经存在，跳过
奶茶多多_7已经存在，跳过
奶茶多多_8已经存在，跳过
奶茶多多_9已经存在，跳过
奶茶多多_10已经存在，跳过
奶茶多多_11已经存在，跳过
生成图片的信息是：Bokeh,(TOK:1.5),  A confident businesswoman in a power suit, surrounded by skyscrapers, she have black long hair,big breasts,big ass,ziprealism
Negative prompt: embedding:negativeXL_D, nsfw,nude,naked
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 5, Seed: 372501187574501, Size: 896x1152, Model hash: d79e975e9b, Model: fudukiMix_v15, Version: ComfyUI
耗时200.26686716079712秒
生成图片的信息是：Bokeh,(TOK:1.5),  A confident businesswoman in a power suit, surrounded by skyscrapers, she have black long hair,big breasts,big ass,ziprealism
Negative prompt: embedding:negativeXL_D, nsfw,nude,naked
Steps: 35, Sampler: dpmpp_2m_sde_karras, CFG Scale: 5, Seed: 841092344846485, Size: 896x1152, Model hash: d79e975e9b, Model: fudukiMix_v15, Version: ComfyUI
耗时107.26703763008118秒
生成图片的信息是：Bokeh,(TOK:1.5),  A confident businesswoman